In [26]:
import matchzoo as mz
import pandas as pd
import numpy as np
import numpy as np 
import tensorflow.keras as K
from matchzoo.preprocessors import BasicPreprocessor
from sklearn.model_selection import train_test_split
import datetime

In [2]:
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import tensorflow as tf
from keras.layers import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam
from random import choice
# from keras_bert import load_trained_model_from_checkpoint, Tokenizer
import re, os
import codecs
from keras.callbacks import Callback
#数据读取及处理
train_left = pd.read_csv('./train/train.query.tsv',sep='\t',header=None)
train_left.columns=['id','q1']
train_right = pd.read_csv('./train/train.reply.tsv',sep='\t',header=None)
train_right.columns=['id','id_sub','q2','label']
df_train = train_left.merge(train_right, how='left')
df_train['q2'] = df_train['q2'].fillna('好的')
test_left = pd.read_csv('./test/test.query.tsv',sep='\t',header=None, encoding='gbk')
test_left.columns = ['id','q1']
test_right =  pd.read_csv('./test/test.reply.tsv',sep='\t',header=None, encoding='gbk')
test_right.columns=['id','id_sub','q2']
df_test = test_left.merge(test_right, how='left')


In [3]:
train_split = int(len(df_train) * 0.7)  # 训练集占训练数据的0.7
dev_split = int(len(df_train) * 0.9)  # 测试集占训练数据的0.2
# test_split = int(len(data_df) * 0.6)  # 验证集占训练数据的0.1

In [4]:
df_train.shape

(21585, 5)

In [5]:
df_test.shape

(53757, 4)

In [6]:
df_train.head(5)

,id,q1,id_sub,q2,label
0,0,采荷一小是分校吧,0,杭州市采荷第一小学钱江苑校区，杭州市钱江新城实验学校。,1
1,0,采荷一小是分校吧,1,是的,0
2,0,采荷一小是分校吧,2,这是5楼,0
3,1,毛坯吗？,0,因为公积金贷款贷的少,0
4,1,毛坯吗？,1,是呢,0


In [7]:
df_test.head(5)

,id,q1,id_sub,q2
0,0,东区西区？什么时候下证？,0,我在给你发套
1,0,东区西区？什么时候下证？,1,您看下我发的这几套
2,0,东区西区？什么时候下证？,2,这两套也是金源花园的
3,0,东区西区？什么时候下证？,3,价钱低
4,0,东区西区？什么时候下证？,4,便宜的房子，一般都是顶楼


In [8]:
# 将训练集和测试集抽取出来，后面再使用训练集分割

In [9]:
sent1_=df_train.q1.values
sent2_=df_train.q2.values
label_=df_train.label.values

In [10]:
sent1_

array(['采荷一小是分校吧', '采荷一小是分校吧', '采荷一小是分校吧', ..., '今天可以安排看房子吗？',
       '今天可以安排看房子吗？', '今天可以安排看房子吗？'], dtype=object)

In [11]:
all_data=pd.DataFrame()
all_data['id_left']=range(len(df_train))
all_data['text_left']=sent1_
all_data['id_right']=range(len(df_train))
all_data['text_right']=sent2_
all_data['label']=label_

In [12]:
all_data.shape

(21585, 5)

In [13]:
_sent1=df_test.q1.values
_sent2=df_test.q2.values
# _label=label[2501:]

In [14]:
tmp_data=pd.DataFrame()
tmp_data['id_left']=range(len(df_test))
tmp_data['text_left']=_sent1
tmp_data['id_right']=range(len(df_test))
tmp_data['text_right']=_sent2
# test_data['label']=_label

In [22]:
# mz.models.list_available()

In [16]:
def load_data(df_data):
# 	df_data = pd.read_csv(data_path, sep='\t', header=None)
# 	df_data = pd.DataFrame(df_data.values, columns=['id_left', 'text_left', 'id_right', 'text_right', 'label'])
	df_data = mz.pack(df_data)
	return df_data

train_data = load_data(all_data)
test_data=load_data(tmp_data)

In [17]:
# preprocessor = BasicPreprocessor(fixed_length_left:int=30, fixed_length_right:int=30),使用最基础的BasicPreprocessor
#将匹配的文本都拓展成15个字符的长度
preprocessor=BasicPreprocessor(15,15)

In [18]:
train = train_data[:train_split]
dev = train_data[train_split:]
train_pack_processed = preprocessor.fit_transform(train) 
# 其实就是做了一个字符转id操作，所以对于中文文本，不需要分词
dev_pack_processed = preprocessor.transform(dev)  
test_pack_processed = preprocessor.transform(test_data)
train_data_generator = mz.DataGenerator(train_pack_processed
                                        , batch_size=32
                                        , shuffle=True)  # 训练数据生成器

test_x, test_y = test_pack_processed.unpack()
dev_x, dev_y = dev_pack_processed.unpack()

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|█| 15109/15109 [00:00<00:00, 16
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|█| 15109/15109 [00:00<00:00, 1
Building Vocabulary from a datapack.: 100%|█████████████████████████████████| 20319/20319 [00:00<00:00, 5077999.34it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|█| 15109/15109 [00:00<00:00, 17
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|█| 15109/15109 [00:00<00:00, 1
Processing text_right with transform: 100%|██████████████████████████████████| 15109/15109 [00:00<00:00, 184256.59it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|█| 6476/6476 [00:00<00:00, 1599
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|█| 6476/6476 [00:00<00:00, 169
Processing text_right with transform: 10

In [20]:
def mkdir(path):
    # 引入模块
    import os
 
    # 去除首位空格
    path=path.strip()
    # 去除尾部 \ 符号
    path=path.rstrip("\\")
 
    # 判断路径是否存在
    # 存在     True
    # 不存在   False
    isExists=os.path.exists(path)
 
    # 判断结果
    if not isExists:
        # 如果不存在则创建目录
        # 创建目录操作函数
        os.makedirs(path) 
 
        print(path+' 创建成功')
        return True
    else:
        # 如果目录存在则不创建，并提示目录已存在
        print(path+' 目录已存在')
        return False

# DenseBaseline

In [26]:
### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print('='*20)
print(task)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
model = mz.models.DenseBaseline()
model.params['task'] = task
model.params['mlp_num_units'] = 3
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
test_x, test_y = test_pack_processed.unpack()
model.fit(x , y,batch_size=32, epochs=10)
print(model.evaluate(dev_x,dev_y))
output_csv=pd.DataFrame()
pred=model.predict(test_x)
output_csv["pred"]=list(pred)

stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join('outputs', 'result','DenseBaseline')
logdir1 = os.path.join('outputs', 'model', 'DenseBaseline')

mkdir(logdir)
output_csv.to_csv('./outputs/result/'+'DenseBaseline'+'/'+stamp+'-pred.csv')

mkdir(logdir1)
### 保存模型

model.save('./outputs/model/'+'DenseBaseline'+'/'+stamp)
print('保存成功')

#这里是加载
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

Ranking Task
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 30)           0           text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            93          concatenate_1[

# DRMMTKS

In [27]:
### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print('='*20)
print(task)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
model =mz.models.DRMMTKS()
model.params['embedding_input_dim'] = 10000
model.params['embedding_output_dim'] = 100
model.params['top_k'] = 20
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 5
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'
model.guess_and_fill_missing_params(verbose=0)
# model.build()
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
test_x, test_y = test_pack_processed.unpack()
model.fit(x , y,batch_size=32, epochs=10)
print(model.evaluate(dev_x,dev_y))
output_csv=pd.DataFrame()
pred=model.predict(test_x)
output_csv["pred"]=list(pred)

stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join('outputs', 'result','DRMMTKS')
logdir1 = os.path.join('outputs', 'model', 'DRMMTKS')

mkdir(logdir)
output_csv.to_csv('./outputs/result/'+'DRMMTKS'+'/'+stamp+'-pred.csv')

mkdir(logdir1)
### 保存模型

model.save('./outputs/model/'+'DRMMTKS'+'/'+stamp)
print('保存成功')

#这里是加载
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

Ranking Task
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 100)      396500      text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dot_1 (Dot)                     (None, 15, 15)       0           embedding[0][0

D:\ancondaRe\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
15109/15109 [==============================] - 4s 260us/step - loss: 0.1816
Epoch 2/10
15109/15109 [==============================] - 4s 241us/step - loss: 0.1585
Epoch 3/10
15109/15109 [==============================] - 4s 240us/step - loss: 0.1432
Epoch 4/10
15109/15109 [==============================] - 4s 241us/step - loss: 0.1378
Epoch 5/10
15109/15109 [==============================] - 4s 237us/step - loss: 0.1336
Epoch 6/10
15109/15109 [==============================] - 4s 237us/step - loss: 0.1269
Epoch 7/10
15109/15109 [==============================] - 4s 251us/step - loss: 0.1192
Epoch 8/10
15109/15109 [==============================] - 4s 242us/step - loss: 0.1136
Epoch 9/10
15109/15109 [==============================] - 4s 240us/step - loss: 0.1109
Epoch 10/10
15109/15109 [==============================] - 4s 241us/step - loss: 0.1088
{mean_average_precision(0.0): 0.24953675108091414}
outputs\result\DRMMTKS 创建成功
outputs\model\DRMMTKS 创建成功
保存成功


# KNRM

In [28]:
### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print('='*20)
print(task)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
model =mz.models.KNRM()
model.params['embedding_input_dim'] =  10000
model.params['embedding_output_dim'] =  10
model.params['embedding_trainable'] = True
model.params['kernel_num'] = 11
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001
model.guess_and_fill_missing_params(verbose=0)
# model.build()

# model.build()
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
test_x, test_y = test_pack_processed.unpack()
model.fit(x , y,batch_size=32, epochs=10)
print(model.evaluate(dev_x,dev_y))
output_csv=pd.DataFrame()
pred=model.predict(test_x)
output_csv["pred"]=list(pred)

stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join('outputs', 'result','KNRM')
logdir1 = os.path.join('outputs', 'model', 'KNRM')

mkdir(logdir)
output_csv.to_csv('./outputs/result/'+'KNRM'+'/'+stamp+'-pred.csv')

mkdir(logdir1)
### 保存模型

model.save('./outputs/model/'+'KNRM'+'/'+stamp)
print('保存成功')

#这里是加载
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

Ranking Task
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 10)       39650       text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dot_3 (Dot)                     (None, 15, 15)       0           embedding[0][0

D:\ancondaRe\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
15109/15109 [==============================] - 1s 53us/step - loss: 127.4504
Epoch 2/10
15109/15109 [==============================] - 1s 38us/step - loss: 2.7715
Epoch 3/10
15109/15109 [==============================] - 1s 39us/step - loss: 0.9134
Epoch 4/10
15109/15109 [==============================] - 1s 38us/step - loss: 0.5450
Epoch 5/10
15109/15109 [==============================] - 1s 38us/step - loss: 0.3912
Epoch 6/10
15109/15109 [==============================] - 1s 38us/step - loss: 0.3472
Epoch 7/10
15109/15109 [==============================] - 1s 38us/step - loss: 0.3547
Epoch 8/10
15109/15109 [==============================] - 1s 38us/step - loss: 0.3687
Epoch 9/10
15109/15109 [==============================] - 1s 38us/step - loss: 0.3782
Epoch 10/10
15109/15109 [==============================] - 1s 38us/step - loss: 0.3132
{mean_average_precision(0.0): 0.24953675108091414}
outputs\result\KNRM 创建成功
outputs\model\KNRM 创建成功
保存成功


# MVLSTM

In [29]:
### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print('='*20)
print(task)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
model =mz.models.MVLSTM()
model.params['lstm_units'] = 32
model.params['top_k'] = 50
model.params['mlp_num_layers'] = 2
model.params['mlp_num_units'] = 20
model.params['mlp_num_fan_out'] = 10
model.params['mlp_activation_func'] = 'relu'
model.params['dropout_rate'] = 0.5
model.guess_and_fill_missing_params(verbose=0)

model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
test_x, test_y = test_pack_processed.unpack()
model.fit(x , y,batch_size=32, epochs=10)
print(model.evaluate(dev_x,dev_y))
output_csv=pd.DataFrame()
pred=model.predict(test_x)
output_csv["pred"]=list(pred)

modeltype='MVLSTM'
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join('outputs', 'result',modeltype)
logdir1 = os.path.join('outputs', 'model',modeltype)

mkdir(logdir)
output_csv.to_csv('./outputs/result/'+modeltype+'/'+stamp+'-pred.csv')

mkdir(logdir1)
### 保存模型

model.save('./outputs/model/'+modeltype+'/'+stamp)
print('保存成功')

#这里是加载
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

Ranking Task
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 300)      1189500     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 15, 64)       85248       embedding[0][0

D:\ancondaRe\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1886
Epoch 2/10
15109/15109 [==============================] - 16s 1ms/step - loss: 0.1657
Epoch 3/10
15109/15109 [==============================] - 16s 1ms/step - loss: 0.1549
Epoch 4/10
15109/15109 [==============================] - 17s 1ms/step - loss: 0.1512
Epoch 5/10
15109/15109 [==============================] - 17s 1ms/step - loss: 0.1490
Epoch 6/10
15109/15109 [==============================] - 17s 1ms/step - loss: 0.1485
Epoch 7/10
15109/15109 [==============================] - 17s 1ms/step - loss: 0.1479
Epoch 8/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1474
Epoch 9/10
15109/15109 [==============================] - 17s 1ms/step - loss: 0.1473
Epoch 10/10
15109/15109 [==============================] - 17s 1ms/step - loss: 0.1476
{mean_average_precision(0.0): 0.24953675108091414}
outputs\result\MVLSTM 创建成功
outputs\model\MVLSTM 创建成功
保存成功


# HBMP

In [30]:
### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print('='*20)
print(task)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
model = mz.contrib.models.HBMP()
model.guess_and_fill_missing_params(verbose=0)
model.params['embedding_input_dim'] = 200
model.params['embedding_output_dim'] = 100
model.params['embedding_trainable'] = True
model.params['alpha'] = 0.1
model.params['mlp_num_layers'] = 3
model.params['mlp_num_units'] = [10, 10]
model.params['lstm_num_units'] = 5
model.params['dropout_rate'] = 0.1
# model.build()

# model.build()
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
test_x, test_y = test_pack_processed.unpack()
model.fit(x , y,batch_size=32, epochs=10)
print(model.evaluate(dev_x,dev_y))
output_csv=pd.DataFrame()
pred=model.predict(test_x)
output_csv["pred"]=list(pred)

modeltype='HBMP'
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join('outputs', 'result',modeltype)
logdir1 = os.path.join('outputs', 'model',modeltype)

mkdir(logdir)
output_csv.to_csv('./outputs/result/'+modeltype+'/'+stamp+'-pred.csv')

mkdir(logdir1)
### 保存模型

model.save('./outputs/model/'+modeltype+'/'+stamp)
print('保存成功')

#这里是加载
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

Ranking Task
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 100)      396500      text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) [(None, 15, 10), (No 4240        embedding[0][0

D:\ancondaRe\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
15109/15109 [==============================] - 22s 1ms/step - loss: 0.1819
Epoch 2/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1535
Epoch 3/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1330
Epoch 4/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1258
Epoch 5/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1206
Epoch 6/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1172
Epoch 7/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1133
Epoch 8/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1107
Epoch 9/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1084
Epoch 10/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1069
{mean_average_precision(0.0): 0.24953675108091414}
outputs\result\HBMP 创建成功
outputs\model\HBMP 创建成功
保存成功


# ArcI

In [21]:
### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print('='*20)
print(task)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
model =mz.models.ArcI()
model.params['num_blocks'] = 1
model.params['left_filters'] = [32]
model.params['right_filters'] = [32]
model.params['left_kernel_sizes'] = [3]
model.params['right_kernel_sizes'] = [3]
model.params['left_pool_sizes'] = [2]
model.params['right_pool_sizes'] = [4]
model.params['conv_activation_func'] = 'relu'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 64
model.params['mlp_num_fan_out'] = 32
model.params['mlp_activation_func'] = 'relu'
model.params['dropout_rate'] = 0.5
model.guess_and_fill_missing_params(verbose=0)
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
test_x, test_y = test_pack_processed.unpack()
model.fit(x , y,batch_size=32, epochs=10)
print(model.evaluate(dev_x,dev_y))
output_csv=pd.DataFrame()
pred=model.predict(test_x)
output_csv["pred"]=list(pred)

modeltype='ArcI'
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join('outputs', 'result',modeltype)
logdir1 = os.path.join('outputs', 'model',modeltype)

mkdir(logdir)
output_csv.to_csv('./outputs/result/'+modeltype+'/'+stamp+'-pred.csv')

mkdir(logdir1)
### 保存模型

model.save('./outputs/model/'+modeltype+'/'+stamp)
print('保存成功')

#这里是加载
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

Ranking Task
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 300)      1189500     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 15, 32)       28832       embedding[0][0

D:\ancondaRe\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
15109/15109 [==============================] - 13s 889us/step - loss: 0.1812
Epoch 2/10
15109/15109 [==============================] - 13s 867us/step - loss: 0.1538
Epoch 3/10
15109/15109 [==============================] - 13s 868us/step - loss: 0.1368
Epoch 4/10
15109/15109 [==============================] - 13s 867us/step - loss: 0.1290
Epoch 5/10
15109/15109 [==============================] - 13s 869us/step - loss: 0.1244
Epoch 6/10
15109/15109 [==============================] - 13s 866us/step - loss: 0.1227
Epoch 7/10
15109/15109 [==============================] - 13s 868us/step - loss: 0.1205
Epoch 8/10
15109/15109 [==============================] - 13s 866us/step - loss: 0.1193
Epoch 9/10
15109/15109 [==============================] - 13s 869us/step - loss: 0.1181
Epoch 10/10
15109/15109 [==============================] - 13s 866us/step - loss: 0.1172
{mean_average_precision(0.0): 0.24953675108091414}
outputs\result\ArcI 创建成功
outputs\model\ArcI 创建成功
保存成功


# ConvKNRM

In [23]:
### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print('='*20)
print(task)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
model = mz.models.ConvKNRM()
model.params['embedding_input_dim'] = 10000
model.params['embedding_output_dim'] = 300
model.params['embedding_trainable'] = True
model.params['filters'] = 128
model.params['conv_activation_func'] = 'tanh'
model.params['max_ngram'] = 3
model.params['use_crossmatch'] = True
model.params['kernel_num'] = 11
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001
model.guess_and_fill_missing_params(verbose=0)
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
test_x, test_y = test_pack_processed.unpack()
model.fit(x , y,batch_size=32, epochs=10)
print(model.evaluate(dev_x,dev_y))
output_csv=pd.DataFrame()
pred=model.predict(test_x)
output_csv["pred"]=list(pred)

modeltype='ConvKNRM'
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join('outputs', 'result',modeltype)
logdir1 = os.path.join('outputs', 'model',modeltype)

mkdir(logdir)
output_csv.to_csv('./outputs/result/'+modeltype+'/'+stamp+'-pred.csv')

mkdir(logdir1)
### 保存模型

model.save('./outputs/model/'+modeltype+'/'+stamp)
print('保存成功')

#这里是加载
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

Ranking Task
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 300)      1189500     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 15, 128)      38528       embedding[0][0

D:\ancondaRe\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
15109/15109 [==============================] - 26s 2ms/step - loss: 2.8632
Epoch 2/10
15109/15109 [==============================] - 23s 2ms/step - loss: 0.4024
Epoch 3/10
15109/15109 [==============================] - 23s 2ms/step - loss: 0.2970
Epoch 4/10
15109/15109 [==============================] - 23s 2ms/step - loss: 0.2393
Epoch 5/10
15109/15109 [==============================] - 23s 2ms/step - loss: 0.2026
Epoch 6/10
15109/15109 [==============================] - 23s 2ms/step - loss: 0.2172
Epoch 7/10
15109/15109 [==============================] - 23s 2ms/step - loss: 0.1891
Epoch 8/10
15109/15109 [==============================] - 24s 2ms/step - loss: 0.1811
Epoch 9/10
15109/15109 [==============================] - 23s 2ms/step - loss: 0.1774
Epoch 10/10
15109/15109 [==============================] - 25s 2ms/step - loss: 0.1747
{mean_average_precision(0.0): 0.24953675108091414}
outputs\result\ConvKNRM 创建成功
outputs\model\ConvKNRM 创建成功
保存成功


# DUET

In [24]:
### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print('='*20)
print(task)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
model = mz.models.DUET()
model.params['embedding_input_dim'] = 1000
model.params['embedding_output_dim'] = 300
model.params['lm_filters'] = 32
model.params['lm_hidden_sizes'] = [64, 32]
model.params['dropout_rate'] = 0.5
model.params['dm_filters'] = 32
model.params['dm_kernel_size'] = 3
model.params['dm_d_mpool'] = 4
model.params['dm_hidden_sizes'] = [64, 32]
model.guess_and_fill_missing_params(verbose=0)
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
test_x, test_y = test_pack_processed.unpack()
model.fit(x , y,batch_size=32, epochs=10)
print(model.evaluate(dev_x,dev_y))
output_csv=pd.DataFrame()
pred=model.predict(test_x)
output_csv["pred"]=list(pred)

modeltype='DUET'
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join('outputs', 'result',modeltype)
logdir1 = os.path.join('outputs', 'model',modeltype)

mkdir(logdir)
output_csv.to_csv('./outputs/result/'+modeltype+'/'+stamp+'-pred.csv')

mkdir(logdir1)
### 保存模型

model.save('./outputs/model/'+modeltype+'/'+stamp)
print('保存成功')

#这里是加载
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

Ranking Task
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 300)      1189500     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 15, 32)       28832       embedding[0][0

D:\ancondaRe\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
15109/15109 [==============================] - 14s 935us/step - loss: 0.1942
Epoch 2/10
15109/15109 [==============================] - 14s 914us/step - loss: 0.1744
Epoch 3/10
15109/15109 [==============================] - 14s 913us/step - loss: 0.1684
Epoch 4/10
15109/15109 [==============================] - 14s 914us/step - loss: 0.1579
Epoch 5/10
15109/15109 [==============================] - 14s 914us/step - loss: 0.1456
Epoch 6/10
15109/15109 [==============================] - 14s 911us/step - loss: 0.1389
Epoch 7/10
15109/15109 [==============================] - 14s 916us/step - loss: 0.1356
Epoch 8/10
15109/15109 [==============================] - 14s 914us/step - loss: 0.1333
Epoch 9/10
15109/15109 [==============================] - 14s 921us/step - loss: 0.1313
Epoch 10/10
15109/15109 [==============================] - 14s 945us/step - loss: 0.1303
{mean_average_precision(0.0): 0.24953675108091414}
outputs\result\DUET 创建成功
outputs\model\DUET 创建成功
保存成功


# ESIM

In [32]:
### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print('='*20)
print(task)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
# model = mz.models.ConvKNRM()
model = mz.contrib.models.ESIM()
model.params['task'] = task
model.params['input_shapes'] = [(15, ), (15, )]
model.params['lstm_dim'] = 300
model.params['mlp_num_units'] = 300
model.params['embedding_input_dim'] =  5000
model.params['embedding_output_dim'] =  10
model.params['embedding_trainable'] = False
model.params['mlp_num_layers'] = 0
model.params['mlp_num_fan_out'] = 300
model.params['mlp_activation_func'] = 'tanh'
model.params['mask_value'] = 0
model.params['dropout_rate'] = 0.5
# model.params['optimizer'] = K.optimizers.Adam(lr=4e-4)
model.guess_and_fill_missing_params()
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
test_x, test_y = test_pack_processed.unpack()
model.fit(x , y,batch_size=32, epochs=10)
print(model.evaluate(dev_x,dev_y))
output_csv=pd.DataFrame()
pred=model.predict(test_x)
output_csv["pred"]=list(pred)

modeltype='ESIM'
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join('outputs', 'result',modeltype)
logdir1 = os.path.join('outputs', 'model',modeltype)

mkdir(logdir)
output_csv.to_csv('./outputs/result/'+modeltype+'/'+stamp+'-pred.csv')

mkdir(logdir1)
### 保存模型

model.save('./outputs/model/'+modeltype+'/'+stamp)
print('保存成功')

#这里是加载
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

Ranking Task
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 10)       39650       text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dropout_15 (Dropout)            multiple             0           embedding[0][0

Epoch 1/10
15109/15109 [==============================] - 206s 14ms/step - loss: nan
Epoch 2/10
15109/15109 [==============================] - 205s 14ms/step - loss: nan
Epoch 3/10
15109/15109 [==============================] - 205s 14ms/step - loss: nan
Epoch 4/10
15109/15109 [==============================] - 205s 14ms/step - loss: nan
Epoch 5/10
15109/15109 [==============================] - 205s 14ms/step - loss: nan
Epoch 6/10
15109/15109 [==============================] - 206s 14ms/step - loss: nan
Epoch 7/10
15109/15109 [==============================] - 205s 14ms/step - loss: nan
Epoch 8/10
15109/15109 [==============================] - 206s 14ms/step - loss: nan
Epoch 9/10
15109/15109 [==============================] - 205s 14ms/step - loss: nan
Epoch 10/10
15109/15109 [==============================] - 206s 14ms/step - loss: nan
{mean_average_precision(0.0): 0.24953675108091414}
outputs\result\ESIM 目录已存在
outputs\model\ESIM 目录已存在
保存成功


# KNRM

In [28]:
### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print('='*20)
print(task)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
# model = mz.models.ConvKNRM()
model =mz.models.KNRM()
model.params['embedding_input_dim'] =  10000
model.params['embedding_output_dim'] =  10
model.params['embedding_trainable'] = True
model.params['kernel_num'] = 11
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001
model.guess_and_fill_missing_params(verbose=0)
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
test_x, test_y = test_pack_processed.unpack()
model.fit(x , y,batch_size=32, epochs=10)
print(model.evaluate(dev_x,dev_y))
output_csv=pd.DataFrame()
pred=model.predict(test_x)
output_csv["pred"]=list(pred)

modeltype='KNRM'
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join('outputs', 'result',modeltype)
logdir1 = os.path.join('outputs', 'model',modeltype)

mkdir(logdir)
output_csv.to_csv('./outputs/result/'+modeltype+'/'+stamp+'-pred.csv')

mkdir(logdir1)
### 保存模型

model.save('./outputs/model/'+modeltype+'/'+stamp)
print('保存成功')

#这里是加载
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

Ranking Task
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 10)       39650       text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dot_13 (Dot)                    (None, 15, 15)       0           embedding[0][0

D:\ancondaRe\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
15109/15109 [==============================] - 1s 57us/step - loss: 552.6871
Epoch 2/10
15109/15109 [==============================] - 1s 42us/step - loss: 31.1491
Epoch 3/10
15109/15109 [==============================] - 1s 43us/step - loss: 3.9013
Epoch 4/10
15109/15109 [==============================] - 1s 41us/step - loss: 1.2155
Epoch 5/10
15109/15109 [==============================] - 1s 41us/step - loss: 0.6865
Epoch 6/10
15109/15109 [==============================] - 1s 41us/step - loss: 0.4989
Epoch 7/10
15109/15109 [==============================] - 1s 42us/step - loss: 0.4319
Epoch 8/10
15109/15109 [==============================] - 1s 41us/step - loss: 0.3658
Epoch 9/10
15109/15109 [==============================] - 1s 40us/step - loss: 0.3145
Epoch 10/10
15109/15109 [==============================] - 1s 41us/step - loss: 0.2736
{mean_average_precision(0.0): 0.24953675108091414}
outputs\result\KNRM 目录已存在
outputs\model\KNRM 目录已存在
保存成功


# MatchLSTM

In [29]:
### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print('='*20)
print(task)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
# model = mz.models.ConvKNRM()
model =mz.contrib.models.MatchLSTM()
model.guess_and_fill_missing_params(verbose=0)
model.params['embedding_input_dim'] = 10000
model.params['embedding_output_dim'] = 100
model.params['embedding_trainable'] = True
model.params['fc_num_units'] = 200
model.params['lstm_num_units'] = 256
model.params['dropout_rate'] = 0.5
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
test_x, test_y = test_pack_processed.unpack()
model.fit(x , y,batch_size=32, epochs=10)
print(model.evaluate(dev_x,dev_y))
output_csv=pd.DataFrame()
pred=model.predict(test_x)
output_csv["pred"]=list(pred)

modeltype='MatchLSTM'
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join('outputs', 'result',modeltype)
logdir1 = os.path.join('outputs', 'model',modeltype)

mkdir(logdir)
output_csv.to_csv('./outputs/result/'+modeltype+'/'+stamp+'-pred.csv')

mkdir(logdir1)
### 保存模型

model.save('./outputs/model/'+modeltype+'/'+stamp)
print('保存成功')

#这里是加载
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

Ranking Task
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 100)      396500      text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
lstm_left (LSTM)                (None, 15, 256)      365568      embedding[0][0

D:\ancondaRe\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
15109/15109 [==============================] - 167s 11ms/step - loss: 0.1866
Epoch 2/10
15109/15109 [==============================] - 167s 11ms/step - loss: 0.1718
Epoch 3/10
15109/15109 [==============================] - 172s 11ms/step - loss: 0.1637
Epoch 4/10
15109/15109 [==============================] - 175s 12ms/step - loss: 0.1621
Epoch 5/10
15109/15109 [==============================] - 176s 12ms/step - loss: 0.1614
Epoch 6/10
15109/15109 [==============================] - 168s 11ms/step - loss: 0.1602
Epoch 7/10
15109/15109 [==============================] - 176s 12ms/step - loss: 0.1601
Epoch 8/10
15109/15109 [==============================] - 177s 12ms/step - loss: 0.1611
Epoch 9/10
15109/15109 [==============================] - 174s 12ms/step - loss: 0.1597
Epoch 10/10
15109/15109 [==============================] - 176s 12ms/step - loss: 0.1595
{mean_average_precision(0.0): 0.24953675108091414}
outputs\result\MatchLSTM 创建成功
outputs\model\MatchLSTM 创建成功
保存成功


# MatchSRNN

In [31]:
### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print('='*20)
print(task)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
# model = mz.models.ConvKNRM()
model =mz.contrib.models.MatchSRNN()
model.params['channels'] = 4
model.params['units'] = 10
model.params['dropout_rate'] = 0.0
model.params['direction'] = 'lt'
model.guess_and_fill_missing_params(verbose=0)
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
test_x, test_y = test_pack_processed.unpack()
model.fit(x , y,batch_size=32, epochs=10)
print(model.evaluate(dev_x,dev_y))
output_csv=pd.DataFrame()
pred=model.predict(test_x)
output_csv["pred"]=list(pred)

modeltype='MatchSRNN'
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join('outputs', 'result',modeltype)
logdir1 = os.path.join('outputs', 'model',modeltype)

mkdir(logdir)
output_csv.to_csv('./outputs/result/'+modeltype+'/'+stamp+'-pred.csv')

mkdir(logdir1)
### 保存模型

model.save('./outputs/model/'+modeltype+'/'+stamp)
print('保存成功')

#这里是加载
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

Ranking Task
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 300)      1189500     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
matching_tensor_layer_1 (Matchi (None, 4, 15, 15)    360000      embedding[0][0

D:\ancondaRe\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
15109/15109 [==============================] - 96s 6ms/step - loss: 0.1799
Epoch 2/10
15109/15109 [==============================] - 94s 6ms/step - loss: 0.1444
Epoch 3/10
15109/15109 [==============================] - 93s 6ms/step - loss: 0.1294
Epoch 4/10
15109/15109 [==============================] - 93s 6ms/step - loss: 0.1209
Epoch 5/10
15109/15109 [==============================] - 94s 6ms/step - loss: 0.1151
Epoch 6/10
15109/15109 [==============================] - 94s 6ms/step - loss: 0.1099: 1s - loss:
Epoch 7/10
15109/15109 [==============================] - 94s 6ms/step - loss: 0.1057: 2s -
Epoch 8/10
15109/15109 [==============================] - 94s 6ms/step - loss: 0.1029
Epoch 9/10
15109/15109 [==============================] - 94s 6ms/step - loss: 0.1009
Epoch 10/10
15109/15109 [==============================] - 94s 6ms/step - loss: 0.0995
{mean_average_precision(0.0): 0.24953675108091414}
outputs\result\MatchSRNN 创建成功
outputs\model\MatchSRNN 创建成功
保存成功


# BiMPM

In [30]:
### 定义任务，包含两种，一个是Ranking，一个是classification
task = mz.tasks.Ranking()
print('='*20)
print(task)

### 创建模型以及修改参数（可以使用mz.models.list_available()查看可用的模型列表）
# model = mz.models.ConvKNRM()
model = mz.contrib.models.BiMPM()
model.guess_and_fill_missing_params(verbose=0)
model.params.update(preprocessor.context)
model.params.completed()
model.build()
model.compile()
model.backend.summary()

### 训练, 评估, 预测
x, y = train_pack_processed .unpack()
test_x, test_y = test_pack_processed.unpack()
model.fit(x , y,batch_size=32, epochs=10)
print(model.evaluate(dev_x,dev_y))
output_csv=pd.DataFrame()
pred=model.predict(test_x)
output_csv["pred"]=list(pred)

modeltype='BiMPM'
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join('outputs', 'result',modeltype)
logdir1 = os.path.join('outputs', 'model',modeltype)

mkdir(logdir)
output_csv.to_csv('./outputs/result/'+modeltype+'/'+stamp+'-pred.csv')

mkdir(logdir1)
### 保存模型

model.save('./outputs/model/'+modeltype+'/'+stamp)
print('保存成功')

#这里是加载
# loaded_model = mz.load_model('./outputs/model/DenseBaseline-model.h5')

Ranking Task
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 300)      1189500     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dropout_10 (Dropout)            (None, 15, 300)      0           embedding[0][0

D:\ancondaRe\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
15109/15109 [==============================] - 20s 1ms/step - loss: 0.1823
Epoch 2/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1523
Epoch 3/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1352
Epoch 4/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1277
Epoch 5/10
15109/15109 [==============================] - 19s 1ms/step - loss: 0.1217
Epoch 6/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1168
Epoch 7/10
15109/15109 [==============================] - 17s 1ms/step - loss: 0.1134
Epoch 8/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1109
Epoch 9/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1081
Epoch 10/10
15109/15109 [==============================] - 18s 1ms/step - loss: 0.1060
{mean_average_precision(0.0): 0.24953675108091414}
outputs\result\BiMPM 创建成功
outputs\model\BiMPM 创建成功
保存成功
